In [1]:
from typing import Dict, List, Tuple
import torch
import numpy as np
import gym
from plotly.subplots import make_subplots
from agent.DQN_Agent import *
from environment.DiscreteEnvironment import DiscreteSimpleEnvironment

%load_ext autoreload
%autoreload 2

## Set environment and agent parameters

In [ ]:
env_arguments = {
"alpha":1,
"beta":1,
"min_temp" : 16,
"max_temp" : 21,
"action_dim": 100,
"modelname":'CELLS_v1.fmu',
"simulation_path":  r'C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\EnergyPlus_simulations\simple_simulation',
"param_list": ['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
"days": 151,  
"hours": 24,  
"minutes": 60,
"seconds": 60,
"ep_timestep": 6
}

env = DiscreteSimpleEnvironment(param_list=['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
alpha=1,
beta=1,
min_temp=16,
max_temp=21,
discrete_action_dim=100,
modelname='CELLS_v1.fmu',
simulation_path=r'C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\EnergyPlus_simulations\simple_simulation',
days=151,
hours=24,
minutes=60,
seconds=60,
ep_timestep=6)

agent_arguments = {
"memory_size": [1000],
"batch_size": [32],
"actor_update":[2],
"target_update": [100],
"epsilon_decay": [1 / 20000],
"max_epsilon": [1],
"min_epsilon":  [0.0],
"lr":[1e-3],
"gamma": [0.99],
"inside_dim": [128],
"num_hidden_layers": [1],
"seed": [778]
}

def cumulative_reward(data:pd.DataFrame) -> float:
    return np.cumsum(np.array(data["Reward"]))[-1]

In [ ]:
from utils import all_combinations_list

best_reward = -np.inf
best_path = ""

logging_path = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\logs\simple_simulation"

for curr_agent_arguments in all_combinations_list(agent_arguments):

    agent = DQNAgent(env).from_dict(dict_arguments=curr_agent_arguments)
    results_path, summary_df = agent.train(logging_path= logging_path, num_episodes=1, num_iterations=5000 ,log=True)
    total_cum_reward = cumulative_reward(summary_df)
    if total_cum_reward > best_reward:
        best_reward = total_cum_reward
        best_path = results_path
  

best_path



In [ ]:
l = [[{'secondary_y': False}], [{'secondary_y': True}], [{'secondary_y': False}], [{'secondary_y': True}], [{'secondary_y': True}], [{'secondary_y': False}], [{'secondary_y': False}], [{'secondary_y': False}]]

len(l)

In [4]:
# Example of how to use the function: 
import Performance

searching_directory = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\logs\simple_simulation\results\DQN_Agent"

conditions={
    "is_test":["=",False],
    "num_iterations": ["=",21744], # to only have have trainings where the full simulation was used
    #"num_episodes":["=",3],
    "alpha":["=",1],
    "beta":["=",1],
    "num_hidden_layers":[">",2]
}

path_list = Performance.search_paths(searching_directory,conditions=conditions, top_k=9)

len(path_list)

[145215.49971531  72248.99796214  72109.20801075  72100.88079197
  72046.07607593  72012.15949514  72000.96291864  71994.13594388
  71914.06701021  71885.19966802  69971.19843831]


9

## Modify jsons

In [ ]:
from pathlib import Path
import json

searching_directory = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\results\DQN_Agent"

for path in Path(searching_directory).glob("**/*json"):
        
        if os.path.getsize(path) > 0 and str(path).__contains__("env_params"):
                f = open(path,"r")
                log_dict = json.load(f)
                closed= False
                if not("discrete_action_dim" in log_dict.keys()) or (log_dict["action_dim"] != 1):
                    print(path)
                    closed=True
                    log_dict["discrete_action_dim"] = log_dict["action_dim"]
                    log_dict["action_dim"] = 1
                    f.close()

                    os.remove(path)
                    with open(path, 'w') as f:
                        f.write(json.dumps(log_dict,indent=True))

                if not(closed):
                    f.close()


# Testing performance pipeline

In [ ]:
import Performance

env = DiscreteSimpleEnvironment(param_list=['Tair', 'RH', 'Tmrt', 'Tout', 'Qheat', 'Occ'],
alpha=1,
beta=1,
min_temp=16,
max_temp=21,
discrete_action_dim=100,
modelname='CELLS_v1.fmu',
simulation_path=r'C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\EnergyPlus_simulations\simple_simulation',
days=151,
hours=24,
minutes=60,
seconds=60,
ep_timestep=6)

agent_arguments = {
"memory_size": [1000],
"batch_size": [32],
"target_update": [100],
"epsilon_decay": [1 / 20000],
"max_epsilon": [1],
"min_epsilon":  [0.0],
"gamma": [0.99],
"inside_dim": [128],
"num_hidden_layers": [1],
"seed": [778]
}


parameter = ("seed", [775,776])

logging_path = r"C:\Users\Harold\Desktop\ENAC-Semester-Project\DIET_Controller\logs\simple_simulation"

utility_function = Performance.cumulative_reward

agent = DQNAgent(env=env)
num_episodes = 1
num_iterations = env.numsteps
agent_name = "DQN_Agent"

dispersion,risk, results_dict = Performance.across_runs(agent=agent, agent_arguments=agent_arguments, parameter=parameter,
    logging_path=logging_path, num_episodes=num_episodes, num_iterations=num_iterations, utility_function=utility_function,alpha=0.05)

logger = SimpleLogger(
        logging_path=logging_path,
        agent_name="DQN_Agent",
        num_episodes=num_episodes,
        num_iterations=num_iterations,
    )

logger.log_performance_pipeline(results_dict)


